In [ ]:
import os
import random
import sys

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from alibi_detect.cd import MMDDrift
import random
from matplotlib.colors import ListedColormap

sys.path.append("..")

from utils.utils import *
from baseline_models.temporal.pytorch.optimizer import Optimizer
from baseline_models.temporal.pytorch.utils import *

In [ ]:
hospital = ["SBK", "UHNTG", "THPC", "THPM", "UHNTW", "SMH","MSH"]
dataset="all"
BASE_DATA_PATH = "/mnt/nfs/project/delirium/drift_exp/risk_of_mortality"

In [ ]:
(x_train, y_train), (x_val, y_val), (x_test, y_test), feats, orig_dims, admin_data = import_dataset_hospital(BASE_DATA_PATH, dataset, hospital, shuffle=True)

In [ ]:
feature_handler = FeatureHandler()
feature_handler.load(BASE_DATA_PATH, "features")

# Get types of columns
numerical_cols = feature_handler.get_numerical_feature_names()["temporal"]
numerical_cols += ["age"]

x_train_normalized = x_train.copy()
x_val_normalized = x_val.copy()
x_test_normalized = x_test.copy()

for col in numerical_cols:
    scaler = StandardScaler().fit(x_train[col].values.reshape(-1, 1))
    x_train_normalized[col] = pd.Series(
        np.squeeze(scaler.transform(x_train[col].values.reshape(-1, 1))),
        index=x_train[col].index,
    )
    x_val_normalized[col] = pd.Series(
        np.squeeze(scaler.transform(x_val[col].values.reshape(-1, 1))),
        index=x_val[col].index,
    )
    x_test_normalized[col] = pd.Series(
        np.squeeze(scaler.transform(x_test[col].values.reshape(-1, 1))),
        index=x_test[col].index,
    )

OUT_DATA_PATH = "/mnt/nfs/project/delirium/drift_exp/datasets"
os.makedirs(os.path.join(OUT_DATA_PATH, dataset), exist_ok=True)

x_train_normalized.to_parquet(
    os.path.join(OUT_DATA_PATH, dataset, "x_train.parquet")
)
x_val_normalized.to_parquet(os.path.join(OUT_DATA_PATH, dataset, "x_val.parquet"))
x_test_normalized.to_parquet(os.path.join(OUT_DATA_PATH, dataset, "x_test.parquet"))

np.save(os.path.join(OUT_DATA_PATH, dataset, "y_train.npy"), y_train)
np.save(os.path.join(OUT_DATA_PATH, dataset, "y_val.npy"), y_val)
np.save(os.path.join(OUT_DATA_PATH, dataset, "y_test.npy"), y_test)

## Create Data Streams

In [ ]:
from datetime import date, timedelta

#####################################################
## a given date contains data from previous two weeks 
#####################################################

start_date = date(2018, 1, 1)
end_date = date(2020, 8, 1)

def daterange(start_date, end_date, stride, window):
    for n in range(int((end_date - start_date).days)):
        if start_date + timedelta(n*stride+window) < end_date:
            yield start_date+ timedelta(n*stride)
            

def get_streams(x, y, admin_data, start_date, end_date, stride, window):
    target_stream_x = []
    target_stream_y = [] 
    measure_dates = []

    admit_df = admin_data[[ENCOUNTER_ID,ADMIT_TIMESTAMP]].sort_values(by=ADMIT_TIMESTAMP)
    for single_date in daterange(start_date, end_date, stride, window):
        if single_date.month ==1 and single_date.day == 1:
            print(single_date.strftime("%Y-%m-%d"),"-",(single_date+timedelta(days=window)).strftime("%Y-%m-%d"))
        encounters_inwindow = admit_df.loc[((single_date+timedelta(days=window)).strftime("%Y-%m-%d") > admit_df[ADMIT_TIMESTAMP].dt.strftime("%Y-%m-%d")) 
                           & (admit_df[ADMIT_TIMESTAMP].dt.strftime("%Y-%m-%d") >= single_date.strftime("%Y-%m-%d")), ENCOUNTER_ID].unique()
        encounter_ids = x.index.get_level_values(0).unique()
        x_inwindow = x.loc[x.index.get_level_values(0).isin(encounters_inwindow)]
        y_inwindow = y[np.in1d(encounter_ids, encounters_inwindow)]
        if not x_inwindow.empty:
            target_stream_x.append(x_inwindow)
            target_stream_y.append(y_inwindow)
            measure_dates.append((single_date+timedelta(days=window)).strftime("%Y-%m-%d"))
    return(target_stream_x, target_stream_y, measure_dates)

x_test_stream, y_test_stream, measure_dates_test = get_streams(x_test_normalized, y_test, admin_data, start_date, end_date, stride=1, window=1)

## Cumulating Rolling Window 

In [ ]:
threshold = 0.05
random.seed(1)
stat_window=14
lookup_window=14
stride=1
#####################################################
## cumulative drift detector 
#####################################################
def cumulative_rolling_window(stat_window, lookup_window,stride, num_timesteps, series, threshold,custom_ref=None):

    p_vals = np.asarray([])
    dist_vals = np.asarray([])
    run_length = int(lookup_window)
    i = stat_window
    
    if custom_ref is not None:
        prev = reshape_inputs(custom_ref, num_timesteps)
        
    while i+stat_window+lookup_window <= len(series):
        feat_index = 0
        
        if custom_ref is None:
            prev = pd.concat(series[max(int(i)-run_length,0):int(i)])
            prev = prev[~prev.index.duplicated(keep='first')]
            prev = reshape_inputs(prev, num_timesteps)
            #prev = prev.reshape(prev.shape[0]*prev.shape[1],prev.shape[2])
            
        next = pd.concat(series[max(int(i)+lookup_window,0):int(i)+stat_window+lookup_window])
        next = next[~next.index.duplicated(keep='first')]
        next = reshape_inputs(next, num_timesteps)
        #next = next.reshape(next.shape[0]*next.shape[1],next.shape[2])
        
        if next.shape[0]<=2 or prev.shape[0]<=2:
            break
            
        ## run distribution shift check here
        cd = MMDDrift(prev, backend='pytorch', p_val=.05)
        preds = cd.predict(next, return_p_val=True, return_distance=True)
        p_val = preds['data']['p_val']
        dist_val = preds['data']['distance']

        #print(max(int(i)-run_length,0),"-", int(i),"-->",max(int(i)+lookup_window,0),"-",int(i)+stat_window+lookup_window,"\tP-Value: ",p_val)
        
        dist_vals = np.concatenate((dist_vals, np.repeat(dist_val, 1)))
        p_vals = np.concatenate((p_vals, np.repeat(p_val, 1)))
        i += stride
        run_length += stat_window

    return dist_vals, p_vals

dist_vals_test, p_vals_test = cumulative_rolling_window(stat_window, lookup_window, stride, num_timesteps, x_test_stream, threshold)

In [ ]:
measure_dates_test_14d = [(datetime.datetime.strptime(date,"%Y-%m-%d")+datetime.timedelta(days=stat_window*2+lookup_window)).strftime("%Y-%m-%d") for date in measure_dates_test]
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(20,8))
fig.suptitle('Cumulative Rolling Windowing')

end = len(p_vals_test)-3
ax1.plot(measure_dates_test_14d[1:end], p_vals_test[1:end], '.-', color="red")
ax1.axhline(y=0.05, color='black', linestyle='--')
ax1.set_ylabel('P-Values')
ax1.set_xticklabels([])
        
ax2.plot(measure_dates_test_14d[1:end], dist_vals_test[1:end], '.-',color="blue")
ax2.set_xlabel('time (s)')
ax2.set_ylabel('Distance Metric')
ax2.tick_params(axis='x', labelrotation=45)
for index, label in enumerate(ax2.xaxis.get_ticklabels()):
    if index % 28 != 0:
        label.set_visible(False)

plt.show()

In [ ]:
stat_window=14
lookup_window=60
stride=1
dist_vals_test, p_vals_test = cumulative_rolling_window(stat_window, lookup_window, stride, num_timesteps, x_test_stream, threshold)

In [ ]:
end = len(p_vals_test)
measure_dates_test_60d = [(datetime.datetime.strptime(date,"%Y-%m-%d")+datetime.timedelta(days=lookup_window+stat_window*2)).strftime("%Y-%m-%d") for date in measure_dates_test]
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(20,8))
fig.suptitle('Cumulative Rolling Windowing')

ax1.plot(measure_dates_test_60d[1:end], p_vals_test[1:end], '.-', color="red")
ax1.axhline(y=0.05, color='black', linestyle='--')
ax1.set_ylabel('P-Values')
ax1.set_xticklabels([])
        
ax2.plot(measure_dates_test_60d[1:end], dist_vals_test[1:end], '.-',color="blue")
ax2.set_xlabel('time (s)')
ax2.set_ylabel('Distance Metric')
ax2.tick_params(axis='x', labelrotation=45)
for index, label in enumerate(ax2.xaxis.get_ticklabels()):
    if index % 28 != 0:
        label.set_visible(False)

plt.show()

## Set constant reference distribution

In [ ]:
threshold = 0.05
num_timesteps = 6
stat_window=14
lookup_window=14
random.seed(1)
dataset2="covid"
(x_train_covid, y_train_covid), (x_val_covid, y_val_covid), (x_test_covid, y_test_covid), feats, orig_dims, admin_data = import_dataset_hospital(BASE_DATA_PATH, dataset2, hospital, shuffle=True)
    
x_train_covid_normalized = x_train_covid.copy()
x_val_covid_normalized = x_val_covid.copy()
x_test_covid_normalized = x_test_covid.copy()

for col in numerical_cols:
    scaler = StandardScaler().fit(x_train_covid[col].values.reshape(-1, 1))
    x_train_covid_normalized[col] = pd.Series(
        np.squeeze(scaler.transform(x_train_covid[col].values.reshape(-1, 1))),
        index=x_train_covid[col].index,
    )
    x_val_covid_normalized[col] = pd.Series(
        np.squeeze(scaler.transform(x_val_covid[col].values.reshape(-1, 1))),
        index=x_val_covid[col].index,
    )
    x_test_covid_normalized[col] = pd.Series(
        np.squeeze(scaler.transform(x_test_covid[col].values.reshape(-1, 1))),
        index=x_test_covid[col].index,
    )

In [ ]:
start_date_covid = date(2019, 1, 1)
end_date_covid = date(2020, 8, 1)
dist_vals_test_covid, p_vals_test_covid = rolling_window(stat_window, lookup_window, num_timesteps, x_test_stream, threshold, x_train_covid)
measure_dates_test_covid = [(datetime.datetime.strptime(date,"%Y-%m-%d")+datetime.timedelta(days=window)).strftime("%Y-%m-%d") for date in measure_dates_test_covid]

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(20,8))
fig.suptitle('Rolling Windowing')

ax1.plot(measure_dates_test_covid[1:len(p_vals_test_covid)], p_vals_test_covid[1:len(p_vals_test_covid)], '.-', color="red")
ax1.axhline(y=0.05, color='black', linestyle='--')
ax1.set_ylabel('P-Values')
ax1.set_xticklabels([])
        
ax2.plot(measure_dates_test_covid[1:len(p_vals_test_covid)], dist_vals_test_covid[1:len(p_vals_test_covid)], '.-',color="blue")
ax2.set_xlabel('time (s)')
ax2.set_ylabel('Distance Metric')
ax2.tick_params(axis='x', labelrotation=45)
for index, label in enumerate(ax2.xaxis.get_ticklabels()):
    if index % 28 != 0:
        label.set_visible(False)

plt.show()